# Solving Double Integrator with SVP

This notebook shows how to use the proposed structured value-based planning (SVP) approach to generate the state-action $Q$-value function for the classic double integrator problem. The correctness of the solution is verified by trajectory simulations.

## Problem Definition

The problem is defined as a unit mass brick moving along the $x$-axis on a frictionless surface, with a control input which provides a horizontal force.
The brick starts from the position-velocity pair $\left(x, \dot{x}\right)$ and follows the dynamics

\begin{align*}
    x &:= x + \dot{x}\cdot\tau, \\
    \dot{x} &:= \dot{x} + u\cdot\tau,
\end{align*}

where $u \in \left[ -1, 1 \right]$ is the acceleration input, $\tau$ is the time interval between decisions. The brick can take on the state values $\left(x, \dot{x}\right) \in \left[ -3., 3. \right] \times \left[ -3., 3. \right]$. To incentivize getting to the original point of the $x$-axis with minimum control input, the reward function is defined in a quadratic form as 

\begin{equation*}
    r(x,\dot{x}) = - \frac{1}{2} \left( x^2 + \dot{x}^2 \right).
\end{equation*}

## Structured Value-based Planning (SVP)

The proposed structured value-based planning (SVP) approach is based on the $Q$-value iteration. At the $t$-th iteration, instead of a full pass over all state-action pairs:
- SVP first randomly selects a subset $\Omega$ of the state-action pairs. In particular, each state-action pair in $\mathcal{S}\times\mathcal{A}$ is observed (i.e., included in $\Omega$) independently with probability $p$. 
- For each selected $(s,a)$, the intermediate $\hat{Q}(s,a)$ is computed based on the $Q$-value iteration: 
    \begin{equation*}\hat{Q}(s,a) \leftarrow \sum_{s'} P(s'|s,a) \left( r(s,a) + \gamma \max_{a'} Q^{(t)}(s',a') \right),\quad\forall\:(s,a)\in\Omega.
    				\end{equation*}
- The current iteration then ends by reconstructing the full $Q$ matrix with matrix estimation, from the set of observations in $\Omega$. That is, $Q^{(t+1)}=\textrm{ME}\big(\{\hat{Q}(s,a)\}_{(s,a)\in\Omega}\big).$

Overall, each iteration reduces the computation cost by roughly $1-p$.

Through SVP, we can compute the final state-action $Q$-value function.
To obtain the optimal policy for state $s$, we compute

\begin{align*}
    \pi^{\star} \left(s\right) = \mbox{argmax}_{a \in \mathcal{A}} Q^{\star}\left(s, a\right).
\end{align*}

### Generate state-action value function with SVP

In [ ]:
push!(LOAD_PATH, ".")
using MDPs, DoubleIntegrator
mdp = MDP(state_space(), action_space(), transition, reward)
__init__()
policy = value_iteration(mdp, true, "../data/qdi_otf_0.2.csv", true)
print("")  # suppress output

### Visualize policy as a heat map

In [2]:
viz_policy(mdp, policy, "SVP policy (20%)", true, "di/policy_di_0.2.tex")

### Verify correctness
Simulate and visualize trajectory from initial state `[position, speed]`.

In [3]:
ss, as = simulate(mdp, policy, [-0.5, 0.0])
viz_trajectory(ss, as, "SVP trajectory (20%)", "SVP input (20%)", true, "di/traj_di_0.2.tex")

In [4]:
nsim = 1000
times = 0
for sim = 1:nsim
    state = [rand(XMIN:0.001 * (XMAX - XMIN):XMAX), 
             rand(VMIN:0.001 * (VMAX - VMIN):VMAX)]
    _, as = simulate(mdp, policy, copy(state))
    times += length(as)
end # for sim
times /= nsim
@printf("average times: %.3f", times)

average times: 199.800

Add `using Printf` to your imports.
  likely near /home/yuzhe/.julia/packages/IJulia/gI2uA/src/kernel.jl:52
